In [54]:
import requests
from io import StringIO
from bs4 import BeautifulSoup
import csv
import re

In [56]:
respuesta = requests.get('https://cuspide.com/100-mas-vendidos/')

In [57]:
respuesta = respuesta.text
soup = BeautifulSoup(respuesta,'html.parser')

In [60]:
def scrape_cuspide_top_books(url):
    # Send a GET request to the URL
    
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    soup = BeautifulSoup(response.text,'html.parser')

    # find the container with the ranking of the 100 books
    book_containers = soup.find_all('div', class_='col small-6 large-2 max-w-large')

    books = [] # data structure where to store the retreived information

    # Then, use soup to extract the books elements from the book list:
    for container in book_containers:
        rank = container.find('p', style='font-size: 12px; text-align: center; line-height: 1.3; margin-bottom: 10px; color: white; padding: 3px; background: #f26636; border-radius: 4px; margin-top: 5px' ).text
        rank = rank.split(' ')[0] # extracting the rank number
        
        title = container.find('h3', class_='name product-title woocommerce-loop-product__title').text.strip()
        
        author = container.find('p', class_='author-product-loop').text.strip()
        
        # Extract the link from the <a> tag inside the <div class="box-image-top">
        link_element = container.find('div', class_='box-image-top').find('a')
        link = link_element['href'] if link_element and 'href' in link_element.attrs else None
        
        price_text = container.find('span', class_='woocommerce-Price-amount amount').text.strip()
        price_numeric = float(re.sub(r'[^0-9]', '', price_text))
        
        # Store the information in a dictionary and # Append  the dictionary to the list
        books.append({"Ranking Position": rank, "Name": title,"Author": author,"Link" : link,"Price": price_numeric})
                    
    return books

In [61]:
scrape_cuspide_top_books('https://cuspide.com/100-mas-vendidos/')

[{'Ranking Position': '#1',
  'Name': 'LA FELICIDAD',
  'Author': 'ROLON, GABRIEL',
  'Link': 'https://cuspide.com/producto/la-felicidad-2/',
  'Price': 1690000.0},
 {'Ranking Position': '#2',
  'Name': 'DESTROZA ESTE DIARIO ( A TODO COLOR )',
  'Author': 'SMITH, KERI',
  'Link': 'https://cuspide.com/producto/destroza-este-diario-a-todo-color/',
  'Price': 2020000.0},
 {'Ranking Position': '#3',
  'Name': 'ZENSORIALMENTE',
  'Author': 'BACHRACH, ESTANISLAO',
  'Link': 'https://cuspide.com/producto/zensorialmente/',
  'Price': 1549900.0},
 {'Ranking Position': '#4',
  'Name': 'HEARTSTOPPER 5',
  'Author': 'OSEMAN, ALICE',
  'Link': 'https://cuspide.com/producto/heartstopper-5/',
  'Price': 1159900.0},
 {'Ranking Position': '#5',
  'Name': 'LA CASA NEVILLE . LA FORMIDABLE SEÑORITA  MANON',
  'Author': 'BONELLI, FLORENCIA',
  'Link': 'https://cuspide.com/producto/la-casa-neville-la-formidable-senorita-manon/',
  'Price': 1970000.0},
 {'Ranking Position': '#6',
  'Name': 'EL VIENTO CONOCE 

Certainly! Let's break down the code step by step:

with open('top_books.csv', 'w', newline='', encoding='utf-8') as csvfile:: This line opens a CSV file named 'top_books.csv' in write mode ('w'). The newline='' parameter ensures that newline characters are handled consistently across different platforms, and encoding='utf-8' specifies the character encoding for the file.

fieldnames = ['Rank', 'Title', 'Author', 'Link', 'Price']: This line defines a list called fieldnames containing the column names that will be used in the CSV file. These column names correspond to the keys in the dictionaries that store book information.

writer = csv.DictWriter(csvfile, fieldnames=fieldnames): This line creates a DictWriter object named writer. This object is designed to write dictionaries to a CSV file, where the keys of the dictionaries correspond to the column names specified in fieldnames.

writer.writeheader(): This line writes the header row to the CSV file. The header row contains the column names ('Rank', 'Title', 'Author', 'Link', 'Price').

for book in all_books:: This line starts a loop that iterates over each book in the all_books list. Each book is represented as a dictionary containing information such as rank, title, author, link, and price.

writer.writerow(book): Within the loop, this line writes a row to the CSV file for each book. The book dictionary is passed to writer.writerow(), and the DictWriter object maps the dictionary keys to the corresponding columns in the CSV file. This effectively writes the book information to the CSV file.

In summary, the code opens a CSV file, writes the header row with column names, and then iterates over a list of dictionaries (representing books) to write each book's information as a row in the CSV file. The resulting CSV file will have a structured format with columns for rank, title, author, link, and price.

In [62]:
# Now we store this dictionary into a .csv file

def main():
    url = 'https://cuspide.com/100-mas-vendidos/'
    
    all_books = []
    
    for page in range(1,6):
        page_url = f'{url}?paged={page}'
        books = scrape_cuspide_top_books(url)
        all_books.extend(books)
        
    # write the data to the .csv file
    
    with open('top_books.csv','w',newline='',encoding='utf-8') as csvfile:
        fieldname = ["Ranking Position", "Name","Author","Link","Price"]
        writer = csv.DictWriter(csvfile,fieldnames=fieldname)
        
        writer.writeheader()
        for book in books:
            writer.writerow(book)
            
if __name__ == '__main__':
    main()


In [63]:
main()

Ahora Mandamos este file a pandas.


In [27]:
import pandas as pd

In [64]:
top_df= pd.read_csv(r'top_books.csv') 
top_df

,Ranking Position,Name,Author,Link,Price
0,#1,LA FELICIDAD,"ROLON, GABRIEL",https://cuspide.com/producto/la-felicidad-2/,1690000.0
1,#2,DESTROZA ESTE DIARIO ( A TODO COLOR ),"SMITH, KERI",https://cuspide.com/producto/destroza-este-dia...,2020000.0
2,#3,ZENSORIALMENTE,"BACHRACH, ESTANISLAO",https://cuspide.com/producto/zensorialmente/,1549900.0
3,#4,HEARTSTOPPER 5,"OSEMAN, ALICE",https://cuspide.com/producto/heartstopper-5/,1159900.0
4,#5,LA CASA NEVILLE . LA FORMIDABLE SEÑORITA MANON,"BONELLI, FLORENCIA",https://cuspide.com/producto/la-casa-neville-l...,1970000.0
...,...,...,...,...,...
95,#96,POLITEIA,"DELGADO, FEDERICO",https://cuspide.com/producto/politeia-2/,1350000.0
96,#97,EL VUELO DE LA LIBELULA,"EXILART, GABRIELA",https://cuspide.com/producto/el-vuelo-de-la-li...,1299900.0
97,#98,TRILOGIA,"FOSSE, JON",https://cuspide.com/producto/trilogia/,1160000.0
98,#99,LA NATURALEZA SECRETA DE LAS COSAS DE ESTE MUNDO,"PRON, PATRICIO",https://cuspide.com/producto/la-naturaleza-sec...,980000.0


In [65]:
top_df['Price'] = top_df['Price']/100

In [66]:
top_df

,Ranking Position,Name,Author,Link,Price
0,#1,LA FELICIDAD,"ROLON, GABRIEL",https://cuspide.com/producto/la-felicidad-2/,16900.0
1,#2,DESTROZA ESTE DIARIO ( A TODO COLOR ),"SMITH, KERI",https://cuspide.com/producto/destroza-este-dia...,20200.0
2,#3,ZENSORIALMENTE,"BACHRACH, ESTANISLAO",https://cuspide.com/producto/zensorialmente/,15499.0
3,#4,HEARTSTOPPER 5,"OSEMAN, ALICE",https://cuspide.com/producto/heartstopper-5/,11599.0
4,#5,LA CASA NEVILLE . LA FORMIDABLE SEÑORITA MANON,"BONELLI, FLORENCIA",https://cuspide.com/producto/la-casa-neville-l...,19700.0
...,...,...,...,...,...
95,#96,POLITEIA,"DELGADO, FEDERICO",https://cuspide.com/producto/politeia-2/,13500.0
96,#97,EL VUELO DE LA LIBELULA,"EXILART, GABRIELA",https://cuspide.com/producto/el-vuelo-de-la-li...,12999.0
97,#98,TRILOGIA,"FOSSE, JON",https://cuspide.com/producto/trilogia/,11600.0
98,#99,LA NATURALEZA SECRETA DE LAS COSAS DE ESTE MUNDO,"PRON, PATRICIO",https://cuspide.com/producto/la-naturaleza-sec...,9800.0


https://brightdata.com/blog/how-tos/web-scraping-with-python